# SABR MODEL

## Variance Swap Under SABR Model

### Libraries

In [102]:
import numpy as np
from scipy.stats import norm

### Hyperparameters

In [104]:
n = 1000
T = 30/252
t = 0
dt = 1/252
m = int((T-t)/dt)

alpha = 0.4
rho = 0.3
# beta = 1 is implicit
S0 = 6025
sigma0 = 0.2
r = 0

### Initialize Vectors

In [106]:
time = np.linspace(t,T,m+1)
St = np.zeros([n,m+1])
vol = np.zeros([n,m+1])

dW1 = np.random.normal(0, np.sqrt(dt),[n,m])
dW2 = np.random.normal(0, np.sqrt(dt),[n,m])
dW2 = rho*dW1 + np.sqrt(1-rho**2)*dW2

int_volW = np.zeros(n)
int_vol2t = np.zeros(n)

### Define Functions

### Set Initial Conditions

In [109]:
St[:,0] = S0
vol[:,0] = sigma0

### Main Logic

In [111]:
for i in range(1, m + 1):
    vol[:, i] = vol[:, i - 1] * np.exp(-0.5 * alpha**2 * dt + alpha * dW2[:, i - 1])
    int_volW += vol[:, i - 1] * dW1[:, i - 1]
    int_vol2t += vol[:, i - 1] ** 2 * dt
    # stock prices are not calculated as they are not needed. only S0 is required.

### Output

In [113]:
print((1/T * np.mean(int_vol2t)))

0.04029602963495427


## VIX Under SABR Model

### Libraries

In [116]:
import pandas as pd
from datetime import date
import yfinance as yf
import numpy as np

In [117]:
### Constants and Variables

In [143]:
K_values = np.linspace(2000,8000,6001)
dK = K_values[1]-K_values[0]
TTM = 30
S0_prime = S0 * np.exp(-0.5 * rho**2 * int_vol2t + rho * int_volW)
vol_eff = np.sqrt((1 - rho**2) / (T - t) * int_vol2t)
K = 6025
K_0 = 6090

### Define Functions

In [146]:
def BlackScholes(S,tau,vol,r,K):
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * tau) / (vol * np.sqrt(tau))
    d2 = d1 - vol * np.sqrt(tau)
    return norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r * tau)
# since we can control how many strikes are available, there is no need to find the midpoint of the bid-ask spread. 
# use the calculated option price instead.
def sum_strikes_simulated(S,Strikes,dK,BS_prices):
    total = 0
    for i in range(1,len(Strikes)-1):
        ksqrd = Strikes[len(Strikes)-i]**2
        qspread = BS_prices[i] if Strikes[i] > S else BS_prices[i]-S+Strikes[i]*np.exp(-r*T) # put/call parity. if r=0, put = call + (K-S)
        total += dK*qspread/ksqrd

### Main Logic

In [155]:
BS_prices = np.zeros(len(K_values))
for k in K_values:
    current_option_price = BlackScholes(S0_prime, T-t, vol_eff, r, k)
    index = np.where(K_values == k)[0]
    BS_prices[index] = np.mean(current_option_price) # computes the expected BS price for this strike and TTM (T-t)
F = K * np.exp(r*TTM) + BS_prices[np.where(Strikes == K)]

### Output

In [158]:
sigsqrd_mar21 = 2*np.exp(r*TTM)*sum_strikes(S0_prime,K_values,dK,BS_prices)/TTM - (1/TTM)*(F/K_0-1)**2
vix_mar21 = np.sqrt(sigsqrd_mar21)*100

NameError: name 'T_mar21' is not defined

# VIX FROM MARKET DATA

### Libraries

In [ ]:
import pandas as pd
from datetime import date
import yfinance as yf
import numpy as np

### Constants and Variables

In [ ]:
S_0 = 6025.99
r = yf.Ticker("^IRX").history(period="1mo")["Close"].iloc[-1]/100

# Read Data. Data taken from Fidelity Investments SPX Options Chain on Feb 7/2025
mar21_am_df = parse_text_file('data/spx_mar21.txt')
mar24_pm_df = parse_text_file('data/spx_mar24.txt')

# Find the time in years between today and expiry
T_mar21 = (date(2025,3,21)-date(2025,2,7)).days/252
T_mar24 = (date(2025,3,24)-date(2025,2,7)).days/252

# the option implied forward price. at EOD friday 7/feb 2025, it is almost
# exactly 6025, which is a strike price. Otherwise, we would interpolate the
# option implied forward value between the two closest strikes
K = 6025
F_mar21 = K*np.exp(r*T_mar21) + (120.1 - 96.1) # from put/call parity
F_mar24 = K*np.exp(r*T_mar24) + (0-99.32) # use BS call price for this missing val?
K_0 = 6090

### Define Functions

In [ ]:
# the structure of this function is as ugly as that of the data it is meant to read
def parse_text_file(file_path):
    # Read the file and process lines
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    # Split into chunks of 11 lines per row
    rows = [lines[i:i+11] for i in range(0, len(lines), 11)]
    
    data = []
    columns = [
        'Call Last', 'Call Change', 'Call Bid', 'Call Ask',
        'Call Volume', 'Call Open Int', 'Call Imp Vol', 'Call Delta', 'Call Action',
        'Strike',
        'Put Action', 'Put Last', 'Put Change', 'Put Bid', 'Put Ask',
        'Put Volume', 'Put Open Int', 'Put Imp Vol', 'Put Delta'
    ]
    
    for row_lines in rows:
        fields = []
        for line in row_lines:
            # Split each line by tabs, strip whitespace, and filter out empty parts
            parts = [part.strip() for part in line.split('\t')]
            parts = [p for p in parts if p]
            fields.extend(parts)
        
        # Check if we have exactly 19 fields
        if len(fields) != 19:
            raise ValueError(f"Expected 19 fields, got {len(fields)} in row: {fields}")
        
        processed = []
        # Process each field according to its column
        try:
            # Call Last
            processed.append(float(fields[0].replace(',', '')))
            # Call Change
            processed.append(float(fields[1].replace(',', '')))
            # Call Bid
            processed.append(float(fields[2].replace(',', '')))
            # Call Ask
            processed.append(float(fields[3].replace(',', '')))
            # Call Volume
            processed.append(int(fields[4].replace(',', '')))
            # Call Open Int
            processed.append(int(fields[5].replace(',', '')))
            # Call Imp Vol
            if fields[6] == "--":
                fields[6] = 0
            else:    
                processed.append(float(fields[6].strip(' %')))
            # Call Delta
            processed.append(float(fields[7]))
            # Call Action
            processed.append(fields[8])
            # Strike
            processed.append(int(fields[9].replace(',', '')))
            # Put Action
            processed.append(fields[10])
            # Put Last
            processed.append(float(fields[11].replace(',', '')))
            # Put Change
            processed.append(float(fields[12].replace(',', '')))
            # Put Bid
            processed.append(float(fields[13].replace(',', '')))
            # Put Ask
            processed.append(float(fields[14].replace(',', '')))
            # Put Volume
            processed.append(int(fields[15].replace(',', '')))
            # Put Open Int
            processed.append(int(fields[16].replace(',', '')))
            # Put Imp Vol
            if fields[17]=="--":
                fields[17] = 0
            else:
                processed.append(float(fields[17].strip(' %')))
            # Put Delta
            processed.append(float(fields[18]))
        except (ValueError, IndexError) as e:
            raise ValueError(f"Error processing row: {fields}") from e
        
        data.append(processed)
    
    df = pd.DataFrame(data, columns=columns)
    return df

def sum_strikes(K_0:float, df):
    total = 0
    kindex = df.index[df["Strike"]==K_0][0]
    strikes = df["Strike"].values 
    delta_k = 0; ksqrd = 0
    for i in range(1,len(strikes)-1):
        print(strikes[i+1],strikes[i-1],i)
        if strikes[i+1] == 'Open or Close Menu.' or strikes[i-1]=='Open or Close Menu.':
            pass
        else:
            delta_k = (strikes[i+1]-strikes[i-1])/2
            ksqrd = strikes[len(strikes)-i]**2
        if i < kindex:
            qspread = (df["Put Bid"].iloc[i]+df["Put Ask"].iloc[i])/2
        elif i > kindex:
            qspread = (df["Call Bid"].iloc[i]+df["Call Ask"].iloc[i])/2
        else:
            qspread = ((df["Put Bid"].iloc[i]+df["Put Ask"].iloc[i])/2 + 
                       (df["Call Bid"].iloc[i]+df["Call Ask"].iloc[i])/2)/2
        total += delta_k*qspread/ksqrd
    return total

### Main Logic

In [ ]:
sigsqrd_mar21 = 2*np.exp(r*T_mar21)*sum_strikes(K_0, mar21_am_df)/T_mar21-(T_mar21**(-1))*(F_mar21/K_0-1)**2
#sigsqrd_mar24 = 2*np.exp(r*T_mar24)*sum_strikes(K_0, mar24_pm_df)/T_mar24

vix_mar21 = np.sqrt(sigsqrd_mar21)*100
print(vix_mar21)